In [57]:
import numpy as np

In [81]:
e_earth = 0.0818191908426 # First eccentricity of earth
g_earth = 9.80665 # Gravity of earth [m/s²]
r_earth = 6378137 # WGS-84 radius of earth [m]
w_earth = 7.2921151467e-5 # Rotation rate of earth [rad/s]

In [99]:
def dn2dlat(dn, lat):
    """
    Convert north-south position (northing) difference to latitude difference.
    
    Arguments:
    - `dn`  : north-south position (northing) difference [m]
    - `lat` : nominal latitude [rad]
    
    Returns:
    - `dlat` : latitude difference [rad]
    """
    
    dlat = dn * np.sqrt(1-(e_earth*np.sin(lat))**2) / r_earth
    
    return dlat

In [100]:
def de2dlon(de, lat):
    """
    Convert east-west position (easting) difference to longitude difference.
    
    Arguments:
    - `de`  : east-west position (easting) difference [m]
    - `lat` : nominal latitude [rad]
    
    Returns:
    - `dlon` : longitude difference [rad]
    """
    
    dlon = de * np.sqrt(1-(e_earth*np.sin(lat))**2) / r_earth / np.cos(lat)
    
    return dlon

In [228]:
def create_P0(lat1=np.radians(45),
              init_pos_sigma = 3.0,
              init_alt_sigma = 0.001,
              init_vel_sigma = 0.01,
              init_att_sigma = np.radians(0.01),
              ha_sigma = 0.001,
              a_hat_sigma = 0.01,
              acc_sigma = 0.000245,
              gyro_sigma = 0.00000000727,
              fogm_sigma = 3.0,
              vec_sigma = 1000.0,
              vec_states = False,
              fogm_state = False,
              P0_TL = np.array([[]])):
    """
    Create P0, initial covariance matrix.
    
    Arguments:
    - `lat1` : initial approximate latitude [rad]
    - `init_pos_sigma` : (optional) initial position uncertainty [m]
    - `init_alt_sigma` : (optional) initial altitude uncertainty [m]
    - `init_vel_sigma` : (optional) initial velocity uncertainty [m]
    - `init_att_sigma` : (optional) initial attitude uncertainty [rad]
    - `ha_sigma` : (optional) barometer aiding altitude bias [m]
    - `a_hat_sigma` : (optional) barometer aiding vertical accel bias [m/s²]
    - `acc_sigma` : (optional) accelerometer bias [m/s²]
    - `gyro_sigma` : (optional) gyroscope bias [rad/s]
    - `fogm_sigma` : (optional) FOGM catch-all bias [nT]
    - `vec_sigma` : (optional) vector magnetometer noise std dev
    - `vec_states` : (optional) if true, include vector magnetometer states
    - `fogm_state` : (optional) if true, include FOGM catch-all bias state
    - `P0_TL` : (optional) initial Tolles-Lawson covariance matrix
    
    Returns:
    - `P0` : initial covariance matrix
    """
    
    nx_TL = np.size(P0_TL,1)
    nx_vec = 3 if vec_states else 0
    nx_fogm = 1 if fogm_state else 0
    
    nx = 17 + nx_TL + nx_vec + nx_fogm
    
    P0 = np.zeros((nx,nx)) # Initial covariance matrix
    
    P0[0,0] = dn2dlat(init_pos_sigma, lat1)**2
    P0[1,1] = de2dlon(init_pos_sigma, lat1)**2
    P0[2,2] = init_alt_sigma**2
    P0[3,3] = init_vel_sigma**2
    P0[4,4] = init_vel_sigma**2
    P0[5,5] = init_vel_sigma**2
    P0[6,6] = init_att_sigma**2
    P0[7,7] = init_att_sigma**2
    P0[8,8] = init_att_sigma**2
    P0[9,9] = ha_sigma**2
    P0[10,10] = a_hat_sigma**2
    P0[11,11] = acc_sigma**2
    P0[12,12] = acc_sigma**2
    P0[13,13] = acc_sigma**2
    P0[14,14] = gyro_sigma**2
    P0[15,15] = gyro_sigma**2
    P0[16,16] = gyro_sigma**2
    
    i1 = 17
    i2 = i1 + nx_TL
    i3 = i2 + nx_vec
    
    if nx_TL > 0: P0[i1:i2-1, i1:i2-1] = P0_TL
    if nx_vec > 0: P0[i2:i3-1, i2:i3-1] = np.diag(np.repeat([vec_sigma**2], nx_vec))
    if nx_fogm > 0: P0[i3, i3] = fogm_sigma**2
        
    return P0

In [229]:
def create_Qd(dt=0.1,
              VRW_sigma = 0.000238,
              ARW_sigma = 0.000000581,
              baro_sigma = 1.0,
              acc_sigma = 0.000245,
              gyro_sigma = 0.00000000727,
              fogm_sigma = 3.0,
              vec_sigma = 1000.0,
              TL_sigma = np.array([[]]),
              baro_tau = 3600.0,
              acc_tau = 3600.0,
              gyro_tau = 3600.0,
              fogm_tau = 600.0,
              vec_states = False,
              fogm_state = True):
    """
    Create Qd, discrete time process/system noise matrix.
    
    Arguments:
    - `dt` : measurement time step [s]
    - `VRW_sigma` : (optional) velocity random walk [m/s² /sqrt(Hz)]
    - `ARW_sigma` : (optional) angular random walk [rad/s /sqrt(Hz)]
    - `baro_sigma` : (optional) barometer bias [m]
    - `acc_sigma` : (optional) accelerometer bias [m/s²]
    - `gyro_sigma` : (optional) gyroscope bias [rad/s]
    - `fogm_sigma` : (optional) FOGM catch-all bias [nT]
    - `vec_sigma` : (optional) vector magnetometer noise std dev
    - `TL_sigma` : (optional) Tolles-Lawson coefficients estimate std dev
    - `baro_tau` : (optional) barometer time constant [s]
    - `acc_tau` : (optional) accelerometer time constant [s]
    - `gyro_tau` : (optional) gyroscope time constant [s]
    - `fogm_tau` : (optional) FOGM catch-all time constant [s]
    - `vec_states` : (optional) if true, include vector magnetometer states
    - `fogm_state` : (optional) if true, include FOGM catch-all bias state
    
    Returns:
    - `Qd` : discrete time process/system noise matrix
    """
    
    VRW_var = VRW_sigma**2  # Velocity random walk noise variance
    ARW_var = ARW_sigma**2  # Angular random walk noise variance
    baro_drive = 2*baro_sigma**2 / baro_tau  # Barometer driving noise
    acc_drive = 2*acc_sigma**2 / acc_tau  # Accelerometer driving noise
    gyro_drive = 2*gyro_sigma**2  # Gyroscope driving noise
    fogm_drive = 2*fogm_sigma**2  # FOGM catch-all driving noise
    TL_var = TL_sigma**2  # Tolles-Lawson coefficient noise variance
    vec_var = vec_sigma**2  # Vector magnetometer noise variance
    
    nx_TL = np.size(TL_sigma,1)
    nx_vec = 3 if vec_states else 0
    nx_fogm = 1 if fogm_state else 0
    
    Q = np.concatenate([np.repeat([1e-30], 3),
         np.repeat([VRW_var], 3),
         np.repeat([ARW_var], 3),
         [baro_drive],
         [1e-30],
         np.repeat([acc_drive], 3),
         np.repeat([gyro_drive], 3),
         np.zeros(nx_TL+nx_vec+nx_fogm)],axis=0)
    
    i1 = 17
    i2 = i1 + nx_TL
    i3 = i2 + nx_vec
    
    if nx_TL > 0: Q[i1:i2-1] = np.reshape(TL_var,(-1,1))
    if nx_vec > 0: Q[i2:i3-1] = np.repeat([vec_var],3)
    if nx_fogm > 0: Q[i3] = fogm_drive

    Qd = np.diag(Q)*dt  # Discrete time process/system noise matrix
    
    return Qd

In [249]:
def get_pinson(nx, lat, vn, ve, vd, fn, fe, fd, Cnb,
               baro_tau = 3600.0,
               acc_tau = 3600.0,
               gyro_tau = 3600.0,
               fogm_tau = 600.0,
               vec_states = False,
               fogm_state = True,
               k1 = 3e-2,
               k2 = 3e-4,
               k3 = 1e-6):
    """
    Get `nx` x `nx` Pinson dynamics matrix. Statse (errors) are:
    
    |Num|State|Units|Description
    |1  |lat  |rad  |latitude
    |2  |lon  |rad  |longitude
    |3  |alt  |m    |altitude
    |4  |vn   |m/s  |north velocity
    |5  |ve   |m/s  |east velocity
    |6  |vd   |m/s  |down velocity
    |7  |tn   |rad  |north tilt (attitude)
    |8  |te   |rad  |east tilt (attitude)
    |9  |td   |rad  |down tilt (attitude)
    |10 |ha   |m    |barometer aiding altitude
    |11 |a_hat|m/s² |barometer aiding vertical acceleration
    |12 |ax   |m/s² |x accelerometer
    |13 |ay   |m/s² |y accelerometer
    |14 |az   |m/s² |z accelerometer
    |15 |gx   |rad/s|x gyroscope
    |16 |gy   |rad/s|y gyroscope
    |17 |gz   |rad/s|z gyroscope
    |end|S    |nT   |FOGM catch-all
    
    Arguments:
    - `nx` : total state dimension
    - `lat` : latitude [rad]
    - `vn` : north velocity [m/s]
    - `ve` : east velocity [m/s]
    - `vd` : down velocity [m/s]
    - `fn` : north specific force [m/s²]
    - `fe` : east specific force [m/s²]
    - `fd` : down specific force [m/s²]
    - `Cnb` : direction cosine matrix (body to navigation)
    - `baro_tau` : (optional) barometer time constant [s]
    - `acc_tau` : (optional) accelerometer time constant [s]
    - `gyro_tau` : (optional) gyroscope time constant [s]
    - `fogm_tau` : (optional) FOGM catch-all time constant [s]
    - `vec_states` : (optional) if true, include vector magnetometer states
    - `fogm_state` : (optional) if true, include FOGM catch-all bias state
    - `k1` : (optional) barometer aiding constant [1/s]
    - `k2` : (optional) barometer aiding constant [1/s²]
    - `k3` : (optional) barometer aiding constant [1/s³]
    
    Returns:
    - `F` : `nx` x `nx` Pinson matrix
    """
    
    tan_1 = np.tan(lat)
    cos_1 = np.cos(lat)
    sin_1 = np.sin(lat)
    
    F = np.zeros((nx,nx))
    
    F[0,2] = -vn / r_earth**2
    F[0,3] = 1 / r_earth
    
    F[1,0] = ve*tan_1 / (r_earth*cos_1)
    F[1,2] = -ve / (cos_1*r_earth**2)
    F[1,4] = 1 / (r_earth*cos_1)
    
    F[2,4] = -k1
    F[2,5] = -1
    F[2,9] = k1
    
    F[3,0] = -ve * (2*w_earth*cos_1 + ve / (r_earth*cos_1**2))
    F[3,2] = (ve**2*tan_1 - vn*vd) / r_earth**2
    F[3,3] = vd / r_earth
    F[3,4] = -2*(w_earth*sin_1 + ve*tan_1 / r_earth)
    F[3,5] = vn / r_earth
    F[3,7] = -fd
    F[3,8] = fe
    
    F[4,0] = 2*w_earth*(vn*cos_1 - vd*sin_1) + vn*ve / (r_earth * cos_1**2)
    F[4,2] = -ve*((vn*tan_1 + vd) / r_earth**2)
    F[4,3] = 2*w_earth*sin_1 + ve * tan_1 / r_earth
    F[4,4] = (vn*tan_1 + vd) / r_earth
    F[4,5] = 2*w_earth*cos_1 + ve / r_earth
    F[4,6] = fd
    F[4,8] = -fn
    
    F[5,0] = 2*w_earth*ve*sin_1
    F[5,2] = (vn**2 + ve**2) / r_earth**2 + k2
    F[5,3] = -2*vn / r_earth
    F[5,4] = -2*(w_earth*cos_1 + ve / r_earth)
    F[5,6] = -fe
    F[5,7] = fn
    F[5,9] = -k2
    F[5,10] = 1
    
    F[6,0] = -w_earth*sin_1
    F[6,2] = -ve**2 / r_earth**2
    F[6,4] = 1 / r_earth
    F[6,7] = -w_earth*sin_1 - ve*tan_1 / r_earth
    F[6,8] = vn / r_earth
    
    F[7,2] = vn / r_earth **2
    F[7,3] = -1 / r_earth
    F[7,6] = w_earth*sin_1 + ve*tan_1 / r_earth
    F[7,8] = w_earth*cos_1 + ve / r_earth
    
    F[8,0] = -w_earth*cos_1 - ve / (r_earth*cos_1**2)
    F[8,2] = ve*tan_1 / r_earth**2
    F[8,4] = -tan_1 / r_earth
    F[8,6] = -vn / r_earth
    F[8,9] = -w_earth*cos_1 - ve / r_earth
    
    F[9,9] = -1 / baro_tau
    
    F[10,2] = k3
    F[10,9] = -k3
    
    F[11,11] = -1 / acc_tau
    F[12,12] = -1 / acc_tau
    F[13,13] = -1 / acc_tau
    F[14,14] = -1 / gyro_tau
    F[15,15] = -1 / gyro_tau
    F[16,16] = -1 / gyro_tau
    
    F[3:5, 11:13] = Cnb
    F[6:8, 14:16] = -Cnb
    
    if vec_states:
        nx_fogm = 1 if fogm_state else 0
        F[-1-nx_fogm-2, -1-nx_fogm-2] = -1e9
        F[-1-nx_fogm-1, -1-nx_fogm-1] = -1e9
        F[-1-nx_fogm, -1-nx_fogm] = -1e9
        
    if fogm_state:
        F[-1, -1] = -1 / fogm_tau
    
    return F

In [250]:
def get_Phi(nx, lat, vn, ve, vd, fn, fe, fd, Cnb, baro_tau, acc_tau, gyro_tau, fogm_tau, dt, vec_states = False, fogm_state = True):
    """
    Get Pinson matrix exponential.
    
    Arguments:
    - `nx` : total state dimension
    - `lat` : latitude [rad]
    - `vn` : north velocity [m/s]
    - `ve` : east velocity [m/s]
    - `vd` : down velocity [m/s]
    - `fn` : north specific force [m/s²]
    - `fe` : east specific force [m/s²]
    - `fd` : down specific force [m/s²]
    - `Cnb` : direction cosine matrix (body to navigation)
    - `baro_tau` : barometer time constant [s]
    - `acc_tau` : accelerometer time constant [s]
    - `gyro_tau` : gyroscope time constant [s]
    - `fogm_tau` : FOGM catch-all time constant [s]
    - `dt` : measurement time step [s]
    - `vec_states` : (optional) if true, include vector magnetometers states
    - `fogm_state` : (optional) if true, include FOGM catch-all bias state
    
    Returns:
    - `Phi` : Pinson matrix exponential
    """
    
    return np.exp(get_pinson(nx, lat, vn, ve, vd, fn, fe, fd, Cnb, baro_tau, acc_tau, gyro_tau, fogm_tau, dt, vec_states = False, fogm_state = True))

In [ ]:
def get_h(itp_mapS, x, lat, lon, alt, date = 2020+185/366, core = False):
    """
    Get expected magnetic measurement using magnetic anomaly map interpolation, FOGM catch-all, and optionally for core magnetic field
    
    Arguments:
    - `itp_mapS` : scalar map grid interpolation
    - `x` : states [lat, lon, ... S]
    - `lat` : latitude [rad]
    - `lon` : longitude [rad]
    - `alt` : altitude [m]
    - `date` : (optional) measurement date for IGRF [yr]
    - `core` : (optional) if true, include core magnetic field measurement
    
    Returns:
    - `h` : expected magnetic measurement [nT]
    """
    
    if core:
        return itp_mapS

In [ ]:
def get_H

In [251]:
# Extend kalman filtered from MagNav.jl rewritten in python
def ekf(lat, lon, alt, vn, ve, vd, fn, fe, fd, Cnb, meas, dt, itp_mapS,
       P0 = create_P0(),
       Qd = create_Qd(),
       R = 1,
       baro_tau = 3600.0,
       acc_tau = 3600.0,
       gyro_tau = 3600.0,
       fogm_tau = 600.0,
       date = 2020+185/366,
       core = False,
       der_mapS = None,
       map_alt = 0):
    """
    Extended Kalman filter (EKF) for airborne magnetic anomaly navigation
    
    Arguments:
    - `lat` : latitude [rad]
    - `lon` : longitude [rad]
    - `alt` : altitude [rad]
    - `vn` : north velocity [m/s]
    - `ve` : east velocity [m/s]
    - `vd` : down velocity [m/s]
    - `fn` : north specific force [m/s²]
    - `fe` : east specific force [m/s²]
    - `fd` : down specific force [m/s²]
    - `Cnb` : direction cosine matrix (body to navigation)
    - `meas` : scalar magnetometer measurement [nT]
    - `dt` : measurement time step [s]
    - `itp_mapS` : scalar map grid interpolation
    - `P0` : (optional) initial covariance matrix
    - `Qd` : (optional) discrete time process/system noise matrix
    - `R` : (optional) measurement (white) noise variance
    - `baro_tau` : (optional) barometer time constant [s]
    - `acc_tau` : (optional) accelerometer time constant [s]
    - `gyro_tau` : (optional) gyroscope time constant [s]
    - `fogm_tau` : (optional) FOGM catch-all time constant [s]
    - `date` : (optional) measurement date for IGRF [yr]
    - `core` : (optional) if true, include core magnetic field in measurement
    - `der_mapS` : (optional) scalar map vertical derivative grid interpolation
    - `map_alt` : (optional) map altitude [m]
    
    Returns:
    - `filt_res` : `FILTres` filter results struct
    """
    
    N = len(lat)
    nx = np.size(P0,1)
    ny = np.size(meas,2)
    x_out = np.zeros((nx,N))
    P_out = np.zeros((nx,nx,N))
    r_out = np.zeros((ny,N))
    
    if len(R) == 2:
        adapt = True
        (R_min,R_max) = R
        R = np.mean(R)
    else:
        adapt = False
        
    x = np.zeros(nx)  # State estimate
    P = P0  # Covariance matrix
    
    for t in range(0,N):
        
        # Pinson matrix exponential
        Phi = get_Phi(nx, lat[t], vn[t], ve[t], vd[t], fn[t], fe[t], fd[t], Cnb[:,:,t], baro_tau, acc_tau, gyro_tau, fogm_tau, dt)
        
        # Measurement residual [ny]
        if (map_alt > 0) and (der_mapS != None):
            resid = meas[t,:] - get_h(itp_mapS, der_mapS, x, lat[t], lon[t], alt[t], map_alt, date=date, core=core)
        else:
            resid = meas[t,:] - get_h(itp_mapS, x, lat[t], lon[t], alt[t], date=date, core=core)
            
        r_out[:,t] = resid
        
        # Measurement Jacobian (repeated gradient here) [ny x nx]
        